# Demo: Object Configurators in mom6_bathy

## Set up a custom regional CESM-MOM6 domain using object configurators

These object configurators include:

1. A horizontal grid creator (Grid Creator)
2. A topography editor (Topo Editor)
3. A vertical grid creator (VGrid Creator)

This notebook demonstrates how to use these object configurators to streamline regional CESM-MOM6 domain generation.

# SECTION 1: Using the Grid Creator

We begin by defining a regional MOM6 domain. To do so, we first generate a horizontal grid.

In [1]:
%load_ext autoreload
%autoreload 2

We instantiate a `Grid` with the necessary parameters, including resolution, xstart, lenx, ystart, leny, and name.

In [2]:
from mom6_bathy.grid import Grid
%matplotlib ipympl
grid = Grid(
  resolution = 0.01,
  xstart = 281.0,
  lenx = 5.0,
  ystart = 35.0,
  leny = 5.0,
  name = "EastCoast",
)

Once we have instantiated a `Grid` object, we are now ready to use the Grid Creator.

## 1. Launching the Grid Creator

First, import and launch the Grid Creator in a notebook cell:

In [8]:
from mom6_bathy.grid_creator import GridCreator
Grid_Creator = GridCreator(grid)
Grid_Creator

GridCreator(children=(VBox(children=(VBox(children=(HTML(value='<h3>Grid Creator</h3>'), FloatSlider(value=0.0…

## 2. Grid Parameter Controls

On the left panel, you’ll find sliders for:

- **Resolution**: Set the grid cell size (degrees).
- **xstart**: Longitude of the lower-left grid corner.
- **lenx**: Length of the grid in the x (longitude) direction (degrees).
- **ystart**: Latitude of the lower-left grid corner.
- **leny**: Length of the grid in the y (latitude) direction (degrees).

Adjust these sliders to change the grid geometry. The map on the right updates automatically.

---

## 3. Resetting the Grid

If you want to revert to the original grid parameters, click the **Reset** button.

---

## 4. Saving and Loading Grids

### Saving

- Enter a **Name** and a **Message** describing your grid.
- Click **Save Grid** to save your current grid to the library.

### Loading

- Use the **Grids** dropdown to select a previously saved grid.
- Click **Load Grid** to load it into the creator.

---

## 5. Grid Library

- The **Grids** dropdown lists all saved grids.
- The details panel shows information about the selected grid (name, date, resolution, size).

---

## 6. Map Features

- The map displays your grid overlaid on coastlines and land.
- A scale bar is shown for reference.
- You can use the toolbar to pan, zoom, and save the map image.

---

## Tips

- Make sure to save your grid before switching to another one.
- Use descriptive names and messages for easy identification.
- All grids are saved in the `Grids` folder under your repository root.
- Note that by rerunning the cell, the Grid Creator reverts back to the original, instantiated Grid in the cell above.
- When you are happy with the `Grid` you have created, make sure to save it! What is shown in the Grid Creator can be used to instantiate a `Topo` object by doing: `grid = Grid_Creator.grid`.

# SECTION 2: Using the Topo Editor

Now that we have created a `Grid`, we can now instantiate a `Topo` object.

In [4]:
from CrocoDash.topo import Topo

topo = Topo(
    grid = Grid_Creator.grid,
    min_depth = 0,
)

In [5]:
bathymetry_name='/glade/derecho/scratch/manishrv/ice/croc/gebco/GEBCO_2024.nc'

topo.interpolate_from_file(
    file_path = bathymetry_name,
    longitude_coordinate_name="lon",
    latitude_coordinate_name="lat",
    vertical_coordinate_name="elevation"
)

If bathymetry setup fails, rerun this function with write_to_file = True
Begin regridding bathymetry...

Original bathymetry size: 16.61 Mb
Regridded size: 6.01 Mb
Automatic regridding may fail if your domain is too big! If this process hangs or crashes,make sure function argument write_to_file = True and,open a terminal with appropriate computational and resources try calling ESMF directly in the input directory None via

`mpirun -np NUMBER_OF_CPUS ESMF_Regrid -s bathymetry_original.nc -d bathymetry_unfinished.nc -m bilinear --src_var depth --dst_var depth --netcdf4 --src_regional --dst_regional`

For details see https://xesmf.readthedocs.io/en/latest/large_problems_on_HPC.html

Afterwards, we run the 'expt.tidy_bathymetry' method to skip the expensive interpolation step, and finishing metadata, encoding and cleanup.



Regridding successful! Now calling `tidy_bathymetry` method for some finishing touches...
setup bathymetry has finished successfully.
Tidy bathymetry: Reading in regri

Once we have instantiated the `Topo` object and run the interpolation step, we can use the Topo Editor.

The **Topo Editor** is a powerful interactive tool for editing, managing, and version-controlling ocean model topography files. This guide will walk you through all major features and workflows.

## 1. Launching the Topo Editor

First, import and launch the Topo Editor in a notebook cell:

In [7]:
from mom6_bathy.topo_editor import TopoEditor

topo.depth["units"] = "m"

Topo_Editor = TopoEditor(topo)
Topo_Editor

TopoEditor(children=(VBox(children=(HTML(value='<h2>Topo Editor</h2>'), VBox(children=(VBox(children=(HTML(val…

## 2. Interface Overview

The Topo Editor interface consists of two main panels:

- **Left Panel:** Control panel for editing, history, snapshots, and version control.
- **Right Panel:** Interactive map displaying the topography.

---

## 3. Display Controls

- **Display Mode:**  
  Use the `Field` toggle to switch between:
  - `depth`: Shows the bathymetry (depth values).
  - `mask`: Shows the land/ocean mask.
  - `basinmask`: Shows basin labels.

- **Min Depth:**  
  Adjust the minimum allowed depth for the domain.

---

## 4. Editing Topography

### Selecting and Editing Cells

- **Select a Cell:**  
  Double-click on a cell in the map to select it. The selected cell will be highlighted, and its coordinates and depth will appear in the control panel.

- **Edit Depth:**  
  Enter a new value in the `Depth (m)` field to change the depth of the selected cell.

### Basin Operations

- **Erase Disconnected Basins:**  
  Removes all basins not connected to the selected cell.

- **Erase Selected Basin:**  
  Removes the entire basin associated with the selected cell.

---

## 5. Undo, Redo, and Reset

- **Undo:**  
  Reverts the last edit.

- **Redo:**  
  Re-applies an undone edit.

- **Reset:**  
  Restores the topography to its original state.

---

## 6. Snapshots and Domains

### Snapshots

- **Save State:**  
  Enter a snapshot name and message, then click `Save State` to save your current edits.

- **Load State:**  
  Use the dropdown to select a previous snapshot and click `Load State` to restore it.

- **Snapshot Details:**  
  The details panel shows information about the selected snapshot.

### Domains

- **Domain Dropdown:**  
  Switch between different domains (grids) using the dropdown and `Switch Domain` button.

---

## 7. Git Version Control

The Topo Editor supports git-based version control for collaborative workflows.

- **Branch Management:**
  - Create a new branch: Enter a name and click `Create Branch`.
  - Delete a branch: Enter the branch name and click `Delete Branch`.
  - Checkout a branch: Select from the dropdown and click `Checkout`.

- **Merging:**
  - Select a source branch and click `Merge Branch` to merge it into the current branch.

---

## 8. Tips and Best Practices

- **Autosave:**  
  The editor autosaves your working state. If you close and reopen, your latest work will be restored.

- **Snapshots:**  
  Take snapshots frequently to save your progress and enable easy rollback.

- **Version Control:**  
  Use branches to experiment with different edits or collaborate with others.

- **Navigation:**  
  Use the map toolbar to zoom, pan, and save images.

- **Cell/Basin Editing:**  
  Use the basin tools to clean up disconnected or unwanted basins for realistic bathymetry.

---

## 9. Example Workflow

1. **Load a Topo file and grid.**
2. **Switch to `depth` mode and inspect the bathymetry.**
3. **Double-click a cell to select and edit its depth.**
4. **Use basin tools to clean up the domain.**
5. **Take a snapshot with a descriptive name and message.**
6. **Create a new branch to try a different editing approach.**
7. **Switch between domains or branches as needed.**
8. **Undo/redo edits if you make a mistake.**
9. **Merge branches when ready.**

---

## 10. Troubleshooting

- If you encounter errors loading or saving, check that your file paths and permissions are correct.
- If git operations fail, ensure your repository is properly initialized and you have the necessary permissions.

# SECTION 3: Using the VGrid Creator

Now that we have created a `Topo`, we can now instantiate a `VGrid` object. Note that we use the maximum depth from the Topo Editor to set the depth of the `VGrid` – this is not strictly necessary.

In [12]:
from mom6_bathy.vgrid import VGrid

vgrid = VGrid.hyperbolic(
    nk = 50,
    depth = Topo_Editor.topo.max_depth,
    ratio=5.0,
    name="EastCoast"
)

## 1. Launching the VGrid Creator

First, import and launch the VGrid Creator in a notebook cell:

In [13]:
from mom6_bathy.vgrid_creator import VGridCreator
VGrid_Creator = VGridCreator(vgrid=vgrid, topo=Topo_Editor.topo) # Setting topo is optional!
VGrid_Creator

VGridCreator(children=(VBox(children=(VBox(children=(HTML(value='<h3>Vertical Grid Creator</h3>'), ToggleButto…

## 2. Interface Overview

The VGrid Creator interface consists of:

- **Left Panel:** Controls for editing vertical grid parameters and managing saved grids.
- **Right Panel:** A plot showing the vertical grid layers.

---

## 3. Editing Vertical Grid Parameters

- **Type:**  
  Choose between `Uniform` (equal layer thickness) and `Hyperbolic` (thickness increases with depth).

- **Levels:**  
  Set the number of vertical layers (`nk`).

- **Depth (m):**  
  Set the total depth of the grid.

- **Top/Bottom Ratio:**  
  For `Hyperbolic` grids, adjust the ratio of the bottom layer thickness to the top layer thickness.  
  *Tip: Hover or adjust the slider to see a help message.*

- **Warning:**  
  If the total depth is less than the maximum depth of your topography, a warning will appear.

- **Reset:**  
  Click to revert to the initial grid parameters.

---

## 4. Plot Panel

- The right panel shows a plot of the vertical grid layers.
- Each horizontal line represents a layer interface.
- The plot updates automatically as you adjust parameters.

---

## 5. Saving and Loading VGrids

### Saving

- Enter a **Name** and a **Message** describing your vertical grid.
- Click **Save VGrid** to save your current grid to the library (`VGrids` folder).

### Loading

- Use the **VGrids** dropdown to select a previously saved grid.
- Click **Load VGrid** to load it into the creator.

---

## 6. VGrid Library

- The **VGrids** dropdown lists all saved vertical grids.
- The details panel shows information about the selected grid (name, date, depth, number of layers).

---

## 7. Tips and Best Practices

- **Choose the right type:**  
  Use `Uniform` for simple cases, `Hyperbolic` for higher resolution near the surface or bottom.

- **Match depth to topo:**  
  Ensure your vertical grid depth is at least as deep as your topography’s maximum depth.

- **Save often:**  
  Save grids with descriptive names and messages for easy identification and reproducibility.

- **Reset if needed:**  
  Use the reset button to quickly revert to your starting configuration.

## You have now generated a regional CESM-MOM6 domain! 